In [1]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd
from io import StringIO

In [2]:
# Create the product review dataset filtering out respective columns


bucket_name = 'personalizeproductreviewdata'
review_data_key = 'amazon_data.csv'



data_s3_location1 = "s3://{}/{}".format(bucket_name, review_data_key)  # S3 URL
product_data = pd.read_csv(data_s3_location1)


product_data = product_data.drop('discounted_price', axis=1)
product_data = product_data.drop('actual_price', axis=1)
product_data = product_data.drop('discount_percentage', axis=1)
product_data = product_data.drop('user_id', axis=1)
product_data = product_data.drop('user_name', axis=1)
product_data = product_data.drop('review_id', axis=1)
product_data = product_data.drop('review_title', axis=1)
product_data = product_data.drop('review_content', axis=1)
product_data = product_data.drop('img_link', axis=1)
product_data = product_data.drop('product_link', axis=1)
product_data = product_data.drop('rating', axis=1)
product_data = product_data.drop('age', axis=1)



product_data.rename(columns={'timestamp': 'CREATION_TIMESTAMP', 'product_id': 'ITEM_ID', 'about_product': 'DESCRIPTION'}, inplace=True)
product_data['CREATION_TIMESTAMP'] = pd.to_datetime(product_data['CREATION_TIMESTAMP']).astype(int) // 10**9  # Convert to Unix timestamp (seconds)






product_data.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,CREATION_TIMESTAMP,ITEM_ID,product_name,category,rating_count,DESCRIPTION
0,1587427700,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,"24,269",High Compatibility : Compatible With iPhone 12...
1,1666858131,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,"43,994","Compatible with all Type C enabled devices, be..."
2,1664695794,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,"7,928",【 Fast Charger& Data Sync】-With built-in safet...
3,1692737458,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...
4,1598981603,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...


In [5]:
# Upload the product data to DynamoDB


session = boto3.Session()
dynamodb = session.resource('dynamodb')

table_name = 'productdata'




try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table {table_name} already exists.")
except dynamodb.meta.client.exceptions.ResourceNotFoundException:
    # Define the table schema
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'ITEM_ID',
                'KeyType': 'HASH'  
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'ITEM_ID',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 100,
            'WriteCapacityUnits':100
        }
    )
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print(f"Table {table_name} created successfully.")


    
product_data['rating_count'] = product_data['rating_count'].astype(str)

# Define the DynamoDB table
table = dynamodb.Table(table_name)

# Insert data into DynamoDB
for index, row in product_data.iterrows():
    item = {
        'timestamp': row['CREATION_TIMESTAMP'],
        'ITEM_ID': row['ITEM_ID'],
        'product_name': row['product_name'],
        'category': row['category'],
        'rating_count': row['rating_count'],
        'description': row['DESCRIPTION']
    }
    
    table.put_item(Item=item)

print("Data inserted successfully.")

Table productdata already exists.
Data inserted successfully.


In [6]:

# Create the user review dataset filtering out respective columns

bucket_name = 'personalizeproductreviewdata'
user_data_key = 'amazon_data.csv'
data_s3_location1 = "s3://{}/{}".format(bucket_name, user_data_key)  # S3 URL
user_data = pd.read_csv(data_s3_location1)


user_data = user_data.drop('rating_count', axis=1)
user_data = user_data.drop('category', axis=1)
user_data = user_data.drop('about_product', axis=1)
user_data = user_data.drop('img_link', axis=1)
user_data = user_data.drop('product_link', axis=1)
user_data = user_data.drop('discounted_price', axis=1)
user_data = user_data.drop('actual_price', axis=1)
user_data = user_data.drop('discount_percentage', axis=1)


user_data.rename(columns={'timestamp': 'TIMESTAMP', 'user_id': 'USER_ID', 'age': 'AGE'}, inplace=True)

user_data['TIMESTAMP'] = pd.to_datetime(user_data['TIMESTAMP']).astype(int) // 10**9  # Convert to Unix timestamp (seconds)
#user_data['rating'] = str(user_data['rating'])

user_data['review_content'] = user_data['review_content'].apply(lambda x: x[:1000] if isinstance(x, str) else x)


user_data.info()
user_data.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   TIMESTAMP       1465 non-null   int64 
 1   product_id      1465 non-null   object
 2   product_name    1465 non-null   object
 3   rating          1465 non-null   object
 4   AGE             1465 non-null   int64 
 5   USER_ID         1465 non-null   object
 6   user_name       1465 non-null   object
 7   review_id       1465 non-null   object
 8   review_title    1465 non-null   object
 9   review_content  1465 non-null   object
dtypes: int64(2), object(8)
memory usage: 114.6+ KB


,TIMESTAMP,product_id,product_name,rating,AGE,USER_ID,user_name,review_id,review_title,review_content
0,1587427700,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,4.2,59,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...
1,1666858131,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,4,38,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...
2,1664695794,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,3.9,58,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a..."
3,1692737458,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,4.2,49,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou..."
4,1598981603,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,4.2,19,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th..."


In [ ]:
user_data.info()

In [7]:
# Read back the user data to DynamoDB

session = boto3.Session()

# Get the DynamoDB resource
dynamodb = session.resource('dynamodb')

# Define the table
table_name = 'userdata'

# Check if the table exists, if not, create it
try:
    table = dynamodb.Table(table_name)
    table.load()
    print(f"Table {table_name} already exists.")
except dynamodb.meta.client.exceptions.ResourceNotFoundException:
    # Define the table schema
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'review_id',
                'KeyType': 'HASH' 
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'review_id',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits':100,
            'WriteCapacityUnits':100
        }
    )

    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print(f"Table {table_name} created successfully.")

table = dynamodb.Table(table_name)
user_data['rating'] = user_data['rating'].astype(str)
user_data['review_content'] = user_data['review_content'].astype(str)



for index, row in user_data.iterrows():
    item = {
        'timestamp': row['TIMESTAMP'],
        'age': row['AGE'],
        'product_id': row['product_id'],
        'review_id': row['review_id'],
        'review_title': row['review_title'],
        'rating': row['rating'],
        'user_id': row['USER_ID'],
        'user_name': row['user_name'],
        'review_content': row['review_content']
    }
    
    table.put_item(Item=item)

print("Data inserted successfully.")





Table userdata created successfully.
Data inserted successfully.


In [8]:
# Perform sentiment analysis on review using Amazon Comprehend

import boto3
import pandas as pd

comprehend = boto3.client('comprehend', region_name='us-west-2')

def chunk_text(text, chunk_size=4000):
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

def get_comprehend_sentiment(text):
    if not text:
        return None, None
    chunks = chunk_text(text)
    sentiments = []
    sentiment_scores = []
    for chunk in chunks:
        response = comprehend.detect_sentiment(Text=chunk, LanguageCode='en')
        sentiment = response['Sentiment']
        sentiment_score = response['SentimentScore'][sentiment.capitalize()]
        sentiments.append(sentiment)
        sentiment_scores.append(sentiment_score)
    avg_sentiment_score = sum(sentiment_scores) / len(sentiment_scores)
    return sentiments[0], avg_sentiment_score

processed_data = user_data.copy()
processed_data['rating'] = pd.to_numeric(processed_data['rating'], errors='coerce')

# Apply sentiment analysis to the 'review_content' column
processed_data[['Sentiment', 'SentimentScore']] = processed_data['review_content'].apply(lambda x: pd.Series(get_comprehend_sentiment(x)))

# Add the EVENT_TYPE column
processed_data['EVENT_TYPE'] = None
processed_data.loc[processed_data['rating'] > 4.0, 'EVENT_TYPE'] = 'read'
processed_data.loc[processed_data['rating'] > 3.0, 'EVENT_TYPE'] = 'click'

# Filter rows that have an EVENT_TYPE assigned
interactions_df = processed_data[processed_data['EVENT_TYPE'].notna()]

# Select relevant columns and rename them
interactions_df = interactions_df[['TIMESTAMP', 'user_name', 'product_id', 'product_name', 'EVENT_TYPE', 'Sentiment', 'SentimentScore']]
interactions_df.rename(columns={
    'timestamp': 'TIMESTAMP',
    'user_name': 'USER_ID',
    'product_id': 'ITEM_ID',
    'product_name': 'ITEM_NAME'
}, inplace=True)

# Print the final DataFrame
print(interactions_df)


       TIMESTAMP                                            USER_ID  \
0     1587427700  Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...   
1     1666858131  ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...   
2     1664695794  Kunal,Himanshu,viswanath,sai niharka,saqib mal...   
3     1692737458  Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...   
4     1598981603  rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...   
...          ...                                                ...   
1460  1677610932  Prabha ds,Raghuram bk,Real Deal,Amazon Custome...   
1461  1695118348  Manu Bhai,Naveenpittu,Evatira Sangma,JAGANNADH...   
1462  1666717829  Nehal Desai,Danish Parwez,Amazon Customer,Amaz...   
1463  1598760958  Shubham Dubey,E.GURUBARAN,Mayank S.,eusuf khan...   
1464  1630448427  Rajib,Ajay B,Vikas Kahol,PARDEEP,Anindya Prama...   

         ITEM_ID                                          ITEM_NAME  \
0     B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1    

In [ ]:
print(user_data)

In [9]:
# read interaction data into S3

from io import StringIO
interactions_filename = "interactions.csv"

interactions_df.info()
print(interactions_df.columns)


csv_buffer = StringIO()
interactions_df.to_csv(csv_buffer, index=False)




s3_client = boto3.client('s3')
s3_client.put_object(Bucket='personalizeproductreviewdata', Key='interactions.csv', Body = csv_buffer.getvalue())


print(interactions_df.dtypes)


<class 'pandas.core.frame.DataFrame'>
Index: 1454 entries, 0 to 1464
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   TIMESTAMP       1454 non-null   int64  
 1   USER_ID         1454 non-null   object 
 2   ITEM_ID         1454 non-null   object 
 3   ITEM_NAME       1454 non-null   object 
 4   EVENT_TYPE      1454 non-null   object 
 5   Sentiment       1454 non-null   object 
 6   SentimentScore  1454 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 90.9+ KB
Index(['TIMESTAMP', 'USER_ID', 'ITEM_ID', 'ITEM_NAME', 'EVENT_TYPE',
       'Sentiment', 'SentimentScore'],
      dtype='object')
TIMESTAMP           int64
USER_ID            object
ITEM_ID            object
ITEM_NAME          object
EVENT_TYPE         object
Sentiment          object
SentimentScore    float64
dtype: object


In [10]:
#Read product data into S3


csv_buffer = StringIO()
product_data.to_csv(csv_buffer, index=False)


s3_client = boto3.client('s3')
s3_client.put_object(Bucket='personalizeproductreviewdata', Key='product_data.csv', Body = csv_buffer.getvalue())




{'ResponseMetadata': {'RequestId': 'J9633PXQR8G7G9V6',
  'HostId': 'DcCoa2VgrQufZR6IfSzF+IYlNKld+VPS/qohXEzj6zt3644wlcNld+/jh+xlyxiMtoDhbmYLGro=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'DcCoa2VgrQufZR6IfSzF+IYlNKld+VPS/qohXEzj6zt3644wlcNld+/jh+xlyxiMtoDhbmYLGro=',
   'x-amz-request-id': 'J9633PXQR8G7G9V6',
   'date': 'Thu, 27 Jun 2024 21:17:14 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f8a5c993e8545d220d3400714fa6219a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"f8a5c993e8545d220d3400714fa6219a"',
 'ServerSideEncryption': 'AES256'}

In [11]:
# read user data into S3

user_data = user_data.drop('review_content', axis=1)


csv_buffer = StringIO()
user_data.to_csv(csv_buffer, index=False)

s3_client = boto3.client('s3')
s3_client.put_object(Bucket='personalizeproductreviewdata', Key='user_data.csv', Body = csv_buffer.getvalue())



{'ResponseMetadata': {'RequestId': 'CXAV06CRPWTQ236R',
  'HostId': '8wUE+4CDFzj/aEr1GXtu7nhkiEjJLaRoGXEEhWahzLFij43SKqYrHEKtngvXA53qfZPF2htofRw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8wUE+4CDFzj/aEr1GXtu7nhkiEjJLaRoGXEEhWahzLFij43SKqYrHEKtngvXA53qfZPF2htofRw=',
   'x-amz-request-id': 'CXAV06CRPWTQ236R',
   'date': 'Thu, 27 Jun 2024 21:17:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e93b7b90434633fee6ca1f655817100d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e93b7b90434633fee6ca1f655817100d"',
 'ServerSideEncryption': 'AES256'}

In [24]:
#Create Dataset

# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-product-recommendations2"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))



{
  "datasetGroupArn": "arn:aws:personalize:us-west-2:151368154323:dataset-group/personalize-product-recommendations2",
  "ResponseMetadata": {
    "RequestId": "cff17a5d-2177-449a-9bf9-c569b2a3335d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:22:55 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "115",
      "connection": "keep-alive",
      "x-amzn-requestid": "cff17a5d-2177-449a-9bf9-c569b2a3335d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [25]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE
CPU times: user 13.8 ms, sys: 3.62 ms, total: 17.4 ms
Wall time: 1min


In [26]:
#create interactions schema


interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ITEM_NAME",
            "type": "string"
        }
    ],
    "version": "1.0"
}



create_schema_response = personalize.create_schema(
    name = "personalize-product-recommendations-interactions",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-product-recommendations-ints",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-west-2:151368154323:schema/personalize-product-recommendations-interactions",
  "ResponseMetadata": {
    "RequestId": "652da158-2732-4cea-aa44-51d8e576c00d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:24:07 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "114",
      "connection": "keep-alive",
      "x-amzn-requestid": "652da158-2732-4cea-aa44-51d8e576c00d",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}
{
  "datasetArn": "arn:aws:personalize:us-west-2:151368154323:dataset/personalize-product-recommendations2/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "62320b0d-ac48-456e-afa2-c1665128f6de",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:24:07 GMT",
    

In [ ]:
user_data.dtypes

In [23]:
product_data.dtypes

CREATION_TIMESTAMP     int64
ITEM_ID               object
product_name          object
category              object
rating_count          object
DESCRIPTION           object
dtype: object

In [28]:

# create user schema
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "product_id",
            "type": "string"
        },
        {
            "name": "product_name",
            "type": "string"
        },
        {
            "name": "rating",
            "type": "string"
        },
        {
            "name": "AGE",
            "type": "int"
        },
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "user_name",
            "type": "string"
        },
        {
            "name": "review_id",
            "type": "string"
        },
        {
            "name": "review_title",
            "type": "string"
        }
    ],
    "version": "1.0"
}


create_users_schema_response = personalize.create_schema(
    name='users-schema15',
    schema=json.dumps(users_schema)
)

users_schema_arn = create_users_schema_response['schemaArn']
print(json.dumps(create_users_schema_response, indent=2))



dataset_type = "Users"
create_dataset_response = personalize.create_dataset(
    name = "personalize-product-recommendations-users1234",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)




users_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-west-2:151368154323:schema/users-schema15",
  "ResponseMetadata": {
    "RequestId": "8bb81064-d0c3-41a3-8735-8db35808d467",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:24:26 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "80",
      "connection": "keep-alive",
      "x-amzn-requestid": "8bb81064-d0c3-41a3-8735-8db35808d467",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}
{
  "datasetArn": "arn:aws:personalize:us-west-2:151368154323:dataset/personalize-product-recommendations2/USERS",
  "ResponseMetadata": {
    "RequestId": "c4161860-367e-4cd4-b06a-bd860a478bb7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:24:26 GMT",
      "content-type": "application/x-amz-json-

In [34]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "CREATION_TIMESTAMP",
            "type": "long"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "product_name",
            "type": ["null", "string"]
        },
        {
            "name": "category",
            "type": ["null", "string"],
            "categorical": True
        },
        {
            "name": "rating_count",
            "type": ["null", "string"]
        },
        {
            "name": "DESCRIPTION",
            "type": ["null", "string"],
            "textual": True
        }
    ],
    "version": "1.0"
}



create_schema_response = personalize.create_schema(
    name="personalize-product-recommendations-items1",
    schema=json.dumps(items_schema)
)

items_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name="personalize-product-recommendations-items-dataset",
    datasetType=dataset_type,
    datasetGroupArn=dataset_group_arn,
    schemaArn=items_schema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))



{
  "schemaArn": "arn:aws:personalize:us-west-2:151368154323:schema/personalize-product-recommendations-items1",
  "ResponseMetadata": {
    "RequestId": "1e39dcec-2bcd-4bcd-8264-abff5565fdd7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:28:07 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "1e39dcec-2bcd-4bcd-8264-abff5565fdd7",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}
{
  "datasetArn": "arn:aws:personalize:us-west-2:151368154323:dataset/personalize-product-recommendations2/ITEMS",
  "ResponseMetadata": {
    "RequestId": "86ed4066-1e7f-4aed-bac1-3d230232c332",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:28:07 GMT",
      "content-ty

In [35]:
s3 = boto3.client('s3')
bucket_name = 'personalizeproductreviewdata'

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}


s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '0BDT9S897RRQ2000',
  'HostId': 'XuOc3K5usXVh3o6zBo/B0/lmyybV001XlgfOQMCoiPAphF4NPMTsoqT+Q0CQZx4/vJoeSO3FUIk=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'XuOc3K5usXVh3o6zBo/B0/lmyybV001XlgfOQMCoiPAphF4NPMTsoqT+Q0CQZx4/vJoeSO3FUIk=',
   'x-amz-request-id': '0BDT9S897RRQ2000',
   'date': 'Thu, 27 Jun 2024 21:28:15 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [37]:
iam = boto3.client("iam")

role_name = "PersonalizeRolePOC1"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::151368154323:role/PersonalizeRolePOC1


In [39]:
# interaction dataset


create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-product-import3",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:151368154323:dataset-import-job/personalize-product-import3",
  "ResponseMetadata": {
    "RequestId": "8825c0b2-1095-4e9d-8865-219ebe932609",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:29:40 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "115",
      "connection": "keep-alive",
      "x-amzn-requestid": "8825c0b2-1095-4e9d-8865-219ebe932609",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [40]:
#user dataset import

user_filename = 'user_data.csv'

create_user_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName="personalize-user-import3",
    datasetArn=users_dataset_arn,
    dataSource={
        "dataLocation": f"s3://{bucket_name}/{user_filename}"
    },
    roleArn=role_arn
)

user_dataset_import_job_arn = create_user_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_user_dataset_import_job_response, indent=2))




{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:151368154323:dataset-import-job/personalize-user-import3",
  "ResponseMetadata": {
    "RequestId": "4a86e8a7-115d-42e6-8554-5eb0c6ccce28",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:29:48 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "112",
      "connection": "keep-alive",
      "x-amzn-requestid": "4a86e8a7-115d-42e6-8554-5eb0c6ccce28",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [41]:
# item dataset import


item_filename = 'product_data.csv'

create_items_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName="personalize-items-import3",
    datasetArn=items_dataset_arn,
    dataSource={
        "dataLocation": f"s3://{bucket_name}/{item_filename}"
    },
    roleArn=role_arn
)

items_dataset_import_job_arn = create_items_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_items_dataset_import_job_response, indent=2))






{
  "datasetImportJobArn": "arn:aws:personalize:us-west-2:151368154323:dataset-import-job/personalize-items-import3",
  "ResponseMetadata": {
    "RequestId": "d7a934f0-3bec-45f1-a4ae-1fbf134b5f29",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:30:12 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "d7a934f0-3bec-45f1-a4ae-1fbf134b5f29",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [44]:
# creating interaction datajob

#%%time
max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE


In [43]:
#create user datajob

max_time = time.time() + 6 * 60 * 60  # 6 hours max wait time
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn=user_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print(f"DatasetImportJob: {status}")
    
    if status == "ACTIVE":
        print("Dataset import job completed successfully.")
        break
    elif status == "CREATE FAILED":
        failure_reason = describe_dataset_import_job_response["datasetImportJob"]['failureReason']
        print(f"Dataset import job failed: {failure_reason}")
        break
    
    time.sleep(60)

DatasetImportJob: ACTIVE
Dataset import job completed successfully.


In [42]:
# create item datajob


max_time = time.time() + 6 * 60 * 60  # 6 hours max wait time
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn=items_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print(f"DatasetImportJob: {status}")
    
    if status == "ACTIVE":
        print("Dataset import job completed successfully.")
        break
    elif status == "CREATE FAILED":
        failure_reason = describe_dataset_import_job_response["datasetImportJob"]['failureReason']
        print(f"Dataset import job failed: {failure_reason}")
        break
    
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
Dataset import job completed successfully.


In [45]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

In [47]:
create_solution_response = personalize.create_solution(
    name = "personalize-soln-user-personalization23",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))



{
  "solutionArn": "arn:aws:personalize:us-west-2:151368154323:solution/personalize-soln-user-personalization23",
  "ResponseMetadata": {
    "RequestId": "2f5b7d15-6f19-470d-963a-c1a73cefdda9",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:36:35 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "109",
      "connection": "keep-alive",
      "x-amzn-requestid": "2f5b7d15-6f19-470d-963a-c1a73cefdda9",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [48]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-west-2:151368154323:solution/personalize-soln-user-personalization23/202e6567",
  "ResponseMetadata": {
    "RequestId": "83fd3492-d357-4c7f-9886-3dd19af827b1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Thu, 27 Jun 2024 21:36:38 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "125",
      "connection": "keep-alive",
      "x-amzn-requestid": "83fd3492-d357-4c7f-9886-3dd19af827b1",
      "strict-transport-security": "max-age=47304000; includeSubDomains",
      "x-frame-options": "DENY",
      "cache-control": "no-cache",
      "x-content-type-options": "nosniff"
    },
    "RetryAttempts": 0
  }
}


In [ ]:
%%time
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-recs3",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.3"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

In [ ]:
# Load the interactions data

items_map = dict(zip(interactions_df['ITEM_ID'].astype(str), interactions_df['ITEM_NAME']))
user_id, item_id = interactions_df[['USER_ID', 'ITEM_ID']].sample().values[0]

print(f"User Id: {user_id.split(',')[0]}")

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn=campaign_arn,
    userId=str(user_id),
)

# Extract recommended item IDs from the response
recommendation_list = [item['itemId'] for item in get_recommendations_response['itemList']]
recommendation_names = [items_map.get(item_id, 'Unknown') for item_id in recommendation_list]


recommendations_df = pd.DataFrame(recommendation_names, columns=['Recommended_Item'])
recommendations_df.head()

In [ ]:
user_ids = interactions_df['USER_ID'].unique()

all_recommendations = []

for user_id in user_ids:
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn=campaign_arn,
        userId=str(user_id)
    )
    
    recommendation_list = [item['itemId'] for item in get_recommendations_response['itemList']]
    
    recommendation_names = [items_map.get(item_id, 'Unknown') for item_id in recommendation_list]
    
    for rec in recommendation_names:
        all_recommendations.append({'User ID': user_id, 'Recommended_Item': rec})


        
recommendations_df = pd.DataFrame(all_recommendations)
csv_filename = 'user_recommendations.csv'
recommendations_df.to_csv(csv_filename, index=False)
print(f"Recommendations DataFrame saved to {csv_filename}")

print(recommendations_df)

In [ ]:
import boto3
import pandas as pd
import time

# Initialize DynamoDB client
dynamodb = boto3.client('dynamodb')

# Create a new table for storing user recommendations
table_name = 'user_recommendations3'  # Replace with your desired table name
dynamodb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            'AttributeName': 'USER_ID',  
            'KeyType': 'HASH'  
        }
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'USER_ID',
            'AttributeType': 'S'  
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 2000, 
        'WriteCapacityUnits': 2000   
    }
)

# make throughput on demand

print(f"Table {table_name} creation initiated. Waiting for table to become active...")

# Wait for the table to become active
waiter = dynamodb.get_waiter('table_exists')
waiter.wait(TableName=table_name)

print(f"Table {table_name} created successfully.")

# Load the user recommendations DataFrame
recommendations_df = pd.read_csv('user_recommendations.csv')

# Insert recommendations into the new table
for index, row in recommendations_df.iterrows():
    user_id = str(row['User ID'])  # Ensure user_id is converted to string
    recommended_item = row['Recommended_Item']
    
    # Insert item into DynamoDB table
    response = dynamodb.put_item(
        TableName=table_name,
        Item={
            'USER_ID': {'S': user_id},
            'Recommended_Item': {'S': recommended_item}
        }
    )
    print(f"Recommendations inserted for user {user_id}: {response}")

print("All recommendations inserted into DynamoDB table.")

In [ ]:
## CONNECT TO BEDROCK

In [ ]:
bedrock = boto3.client('bedrock-runtime')

# Model parameters
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
max_tokens = 1000




def generate_personalize_simple_prompt(product_list, model_id, max_tokens_to_sample=1000):
    prompt_template = f'''
Human: Write a marketing email advertising several products available in our store, given the product recommendations below. The products to recommend and their information is contained in the <product> tag. Put the email between <email> tags.

<product>
{product_list}
</product>

Assistant: Email body:
<email>.
Assistant:
'''
    if 'claude' in model_id:
        prompt_input = json.dumps({"prompt": prompt_template, "max_tokens": max_tokens_to_sample, "anthropic_version": "bedrock-2023-05-31"})
    
    return prompt_input



In [ ]:
def getPersonalizedEmail(bedrock_client, model_id, max_tokens_to_sample, prompt ):
    
    personalized_email = "ERROR"
    
    body = json.dumps({
      "max_tokens": max_tokens_to_sample,
      "messages": [{"role": "user", "content": prompt}],
      "anthropic_version": "bedrock-2023-05-31"
    })
    
    response = bedrock.invoke_model(body=body, modelId=model_id)
    
    response_body = json.loads(response.get('body').read())
        
    # Clean Gen AI response
    personalized_email = re.sub(r'<[^>]*>', '', response_body['content'][0]['text'])

    
    return personalized_email

In [ ]:
recommendations_df = pd.read_csv('user_recommendations.csv')

items_map = dict(zip(recommendations_df['Recommended_Item'].astype(str), recommendations_df['Recommended_Item']))

# Function to generate an email for each user
def generate_email_for_user(user_id, recommendations):
    recommendations_str = "\n".join(recommendations)
    prompt_input_json = generate_personalize_simple_prompt(recommendations_str, model_id, max_tokens)
    
    email_content = getPersonalizedEmail(bedrock, model_id, max_tokens_to_sample, prompt_input_json)
    
    return email_content
'''
# Iterate over users and generate emails
user_recommendations = recommendations_df.groupby('User ID')['Recommended_Item'].apply(list).reset_index()

emails = []
for index, row in user_recommendations.iterrows():
    user_id = row['User ID']
    recommendations = row['Recommended_Item']
    email_content = generate_email_for_user(user_id, recommendations)
    emails.append({'User ID': user_id, 'Email Content': email_content})

# Convert to DataFrame for easier handling
emails_df = pd.DataFrame(emails)

# Display the generated emails
print(emails_df)

# Save to CSV if needed
emails_df.to_csv('user_emails.csv', index=False)
print("Emails saved to 'user_emails.csv'")

'''